In [ ]:
from modules.segmentation_model import Unet
from torchinfo import summary
import torch

In [ ]:
model = Unet(
    in_channels=1,
    depths=[3, 3, 12, 3],
    channel_multipliers=[1, 2, 4, 8],
    embed_dim=12,
    classes=1,
)


In [ ]:
print(model)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
summary(model, input_size=(1, 1, 128, 128, 128), depth=5)

In [ ]:
print(model)